In [1]:
from bs4 import BeautifulSoup
import pandas as pd
from tqdm import tqdm_notebook
from urllib.request import Request, urlopen

In [2]:
# 성분 파일(csv) 불러오기
ing_df = pd.read_csv('../data/ing_name.csv',index_col=0)

In [5]:
# 성분 파일에서 'formatted_영문명' 컬럼만 웹페이지 접근용으로 사용
# 결측값 제거 후 리스트로 변환
ing_list = list(ing_df['formatted_영문명'].dropna())

product_name = set() # 제품명
product_label = set() # 제품명 (formatted - 웹페이지 접근용)
search_failed = [] # 'formatted_영문명' 값으로 웹페이지 접근이 불가했던 건들 확인용도

----

FUNCTIONS

In [3]:
# 성분으로 조회한 제품 리스트 만들기
def add_ing_products(tags):  # html tag 를 받아와 조회하여 
    for tag in tags:
        if tag.text not in product_name: # 중복된 데이터는 추가하지 않도록, tag의 제품명이 product_name 셋에 없는 경우에만 추가
            product_name.add(tag.text)
            product_label.add(tag.attrs['data-ga-eventlabel'][8:])

In [4]:
# 성분으로 접근한 웹페이지의 제품 리스트에 '다음페이지'가 존재하는지 확인하기 
def next_page_exists(soup):
    if "Next" in soup.find(id="product").find_all("div")[-1].text: # Next라는 문자가 해당 태그안에 존재하는지 여부 확인
        return True
    else:
        return False

----

CRAWLING - REMARK<br>
성분페이지로 접근을 시도할 성분이 19417개이며, 개중 여러 제품에 공통적으로 들어가는 성분(예: water)의 경우 제품 페이지가 최대 1342페이지 까지 있음<br>
&rarr; 데이터 수집에 상당한 시간이 소요 (24시간 이상 예상)

아래 소스코드를 사용하되, 효율적인 수집을 위하여 다음과 같이 진행함:
1. 성분리스트(ing_list)를 분할 (list slicing)<br> 
   &rarr; 다수의 컴퓨터에서 동시 수집하여 저장 후 병합

2. 여러 제품에 공통적으로 들어가는 성분(water, glycerin)은 성분페이지 접근과정에서 제외<br>
   (최종적으로 사용할 성분데이터는 InciDecoder 제품페이지에서 수집할 "주성분 테이블"의 데이터이기 때문)


In [ ]:
for ing in tqdm_notebook(ing_list): # 성분 리스트의 각 성분(formatted) 사용 
    # 실제 수집시에는 슬라이싱으로 분할 수집 (예: ing_list[:2001],ing_list[2001:3001],...)
    
    url = 'https://incidecoder.com/ingredients/'+ ing # url 주소를 생성
    
    # if page exists (url로 접근 가능시)
    try:
        html = urlopen(url) 
        source = html.read()
        soup = BeautifulSoup(source, "html.parser")
        tags = soup.select("#product > div > a") # html의 태그 불러오기  
        add_ing_products(tags) # 태그에서 제품명 (일반+formatted) 리스트에 저장 - 중복건은 추가 x

        if next_page_exists(soup): # 제품리스트가 1페이지 이상인지 확인
            nextpage = True
        
        while nextpage: # 다음페이지가 존재하는 경우 반복 
            nexturl = soup.find(id="product").find_all("a")[-1]['href'] # href태그로 다음페이지 url을 생성하여 해당 페이지 접근
            url = 'https://incidecoder.com'+ nexturl 
            html = urlopen(url) 
            source = html.read()
            soup = BeautifulSoup(source, "html.parser")
            tags = soup.select("#product > div > a")
            add_ing_products(tags) # 다음페이지에서도 동일하게 제품명 받아와서 저장

            if not next_page_exists(soup): # 더이상 다음 페이지가 없는 경우 while문 빠져나옴 
                nextpage = False
    
    # if page does NOT exist (url로 접근 불가시 추후 확인 용도로 search_failed 리스트에 추가)
    except Exception:
        search_failed.append(ing)
        pass 

In [ ]:
product_all = pd.DataFrame(columns=['product_label'])
product_all['product_label'] = list(product_label)
product_all.to_csv('../data/product_list.csv',index=False)

In [ ]:
product_all.info()

---

실제 수집 시에는 상기 REMARK에 명시한 바와 같이 분할 수집하여 아래 과정을 통해 병합

In [ ]:
# product_list1 = pd.read_csv('product_list1.csv',index_col=False)
# product_list2 = pd.read_csv('product_list2.csv',index_col=False)
# product_list3 = pd.read_csv('product_list3.csv',index_col=False)
# product_list4 = pd.read_csv('product_list4.csv',index_col=False)
# product_list5 = pd.read_csv('product_list5.csv',index_col=False)
# product_list6 = pd.read_csv('product_list6.csv',index_col=False)
# product_list7 = pd.read_csv('product_list7.csv',index_col=False)
# product_list8 = pd.read_csv('product_list8.csv',index_col=False)
# product_list9 = pd.read_csv('product_list9.csv',index_col=False)

# product_all = pd.concat([product_list1,product_list2,product_list3,product_list4,product_list5,product_list6,product_list7,product_list8,product_list9],ignore_index=True).drop_duplicates()
# product_all['product_label'] = list(product_label)
# product_all.to_csv('../data/product_list.csv',index=False)